In [1]:
import requests

# Replace with your Page Access Token and Page ID
ACCESS_TOKEN = 'EAABwzLix5rIBANen1234567890ilgvdttjkliodhtjggujlouoi987ghfh976dghgki89o90jli9iojllkkmen0987654321ABjklmnopQRSTUVwxYZ'
PAGE_ID = 'Costco'  # Replace with the Costco page username or ID
BASE_URL = 'https://graph.facebook.com/v16.0'

# Function to fetch posts with pagination
def fetch_all_posts(page_id, access_token):
    url = f"{BASE_URL}/{page_id}/posts"
    params = {
        'access_token': access_token,
        'fields': 'id,message,created_time',
        'limit': 10  # Number of posts per page
    }
    posts = []

    while url:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            posts.extend(data.get('data', []))  # Add current batch of posts
            
            # Get the next page URL
            url = data.get('paging', {}).get('next', None)
        else:
            print(f"Error fetching posts: {response.status_code}, {response.text}")
            break

    return posts

# Function to fetch all comments for a post with pagination
def fetch_all_comments(post_id, access_token):
    url = f"{BASE_URL}/{post_id}/comments"
    params = {
        'access_token': access_token,
        'fields': 'id,message,created_time,from',
        'limit': 10  # Number of comments per page
    }
    comments = []

    while url:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            data = response.json()
            comments.extend(data.get('data', []))  # Add current batch of comments
            
            # Get the next page URL
            url = data.get('paging', {}).get('next', None)
        else:
            print(f"Error fetching comments for post {post_id}: {response.status_code}, {response.text}")
            break

    return comments

# Fetch posts and their comments
def main():
    print("Fetching posts...")
    posts = fetch_all_posts(PAGE_ID, ACCESS_TOKEN)
    if not posts:
        print("No posts found.")
        return

    for post in posts:
        print(f"\nPost ID: {post['id']}, Message: {post.get('message', 'No message')}, Created: {post['created_time']}")

        print("Fetching comments...")
        comments = fetch_all_comments(post['id'], ACCESS_TOKEN)
        for comment in comments:
            print(f"  Comment ID: {comment['id']}, Message: {comment['message']}, From: {comment['from']['name']}, Created: {comment['created_time']}")

if __name__ == "__main__":
    main()


Fetching posts...
Error fetching posts: 400, {"error":{"message":"(#10) This endpoint requires the 'pages_read_engagement' permission or the 'Page Public Content Access' feature. Refer to https:\/\/developers.facebook.com\/docs\/apps\/review\/login-permissions#manage-pages and https:\/\/developers.facebook.com\/docs\/apps\/review\/feature#reference-PAGES_ACCESS for details.","type":"OAuthException","code":10,"fbtrace_id":"A44-JF8zMulcTyWCNt9Prg-"}}
No posts found.


In [1]:
import requests

ACCESS_TOKEN = 'EAABwzLix5rIBANen1234567890ilgvdttjkliodhtjggujlouoi987ghfh976dghgki89o90jli9iojllkkmen0987654321ABjklmnopQRSTUVwxYZ'
PAGE_ID = 'Costco'  # Replace with the page username or ID
BASE_URL = f'https://graph.facebook.com/v16.0/{PAGE_ID}/posts'

response = requests.get(BASE_URL, params={'access_token': ACCESS_TOKEN})

if response.status_code == 200:
    print(response.json())
else:
    print(f"Error: {response.status_code}, {response.text}")


Error: 400, {"error":{"message":"(#10) This endpoint requires the 'pages_read_engagement' permission or the 'Page Public Content Access' feature. Refer to https:\/\/developers.facebook.com\/docs\/apps\/review\/login-permissions#manage-pages and https:\/\/developers.facebook.com\/docs\/apps\/review\/feature#reference-PAGES_ACCESS for details.","type":"OAuthException","code":10,"fbtrace_id":"AfqkzlQpGThaNHUzf_ZT51e"}}


In [2]:
import requests
from bs4 import BeautifulSoup

def scrape_costco_product_urls(base_url, pages_to_scrape=1):
    product_urls = []

    for page in range(1, pages_to_scrape + 1):
        url = f"{base_url}?page={page}"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Failed to fetch page {page}. Status code: {response.status_code}")
            continue

        soup = BeautifulSoup(response.content, 'html.parser')

        # Locate product elements
        product_elements = soup.find_all('div', {'data-testid': 'product'})
        for product in product_elements:
            link_tag = product.find('a', href=True)
            if link_tag:
                product_url = link_tag['href']
                if product_url.startswith('/'):
                    product_url = 'https://www.costco.com' + product_url
                product_urls.append(product_url)

    return product_urls

if __name__ == "__main__":
    base_url = "https://www.costco.com/grocery-household.html"  # Update as needed
    pages_to_scrape = 5  # Number of pages you want to scrape

    urls = scrape_costco_product_urls(base_url, pages_to_scrape)
    print(f"Scraped {len(urls)} product URLs:")
    for url in urls:
        print(url)


ReadTimeout: HTTPSConnectionPool(host='www.costco.com', port=443): Read timed out. (read timeout=None)